# Melding Unsharp Masking with Morphological Operators
Author: Nana Owusu

Purpose: To generate a mask of fMRI images of NIfTI format by
way of unsharp masking and mathematical morphology operators.
This is an attempt at generatinig a better mask

In [ ]:
# Module for opening the image
# import matplotlib.image as mpimg
# from PIL import Image
# from pydicom import dcmread
# from operator import itemgetter
from nibabel import load, Nifti1Image, save
from glob import glob
from math import floor

# Module for showing the image
import matplotlib.pyplot as plt

# Module for image morphology operators
import numpy as np 
# from skimage.measure import regionprops
from scipy.ndimage import binary_erosion, binary_dilation, grey_erosion, \
generate_binary_structure, binary_fill_holes, label, sum, convolve1d
from scipy.signal.windows import general_gaussian

## Custom Functions

In [ ]:
def multi_slice_display(input_vol):
    """Display all the slices of a single
    timepoint"""
    
    _, _, o = input_vol.shape
    
    # Instantiate a Figure object with one row
    # and multiple columns of axes.
    fig, axes = plt.subplots(1, o, figsize=[28, 35])
    fig.subplots_adjust(wspace=0.125)
    
    # Fill the figure with each slice
    for i,ax in enumerate(axes.flat):
        ax.imshow(np.rot90(input_vol[:,:,i]), cmap='gray')
        ax.set_axis_off()
        
    plt.show

In [ ]:
class struct_el(object):
    
    def __init__(self):
        self.shape = \
        {'square': self.square,
         'cube': self.cube,
         'circle': self.circle,
         'sphere': self.sphere,
         '4neighbor': self.neighbor4,
         '8neighbor': self.neighbor8
        }
        
    def neighbor8(self, d):
        struct = np.zeros((5, 5), dtype='uint8')
        struct[1:4,:] = 1
        struct[:,1:4] = 1
        if d == 0:
            return struct.astype(np.bool)
        elif d is not 0:                
            buffer = [struct.copy() for _ in range(d)]
            new_struct = np.stack(buffer, axis=0)
            return new_struct.astype(np.bool)
        
    def neighbor4(self, d):
        struct = np.zeros((3, 3), dtype='uint8')
        struct[:,1] = 1
        struct[1,:] = 1
        if d == 0:
            return struct.astype(np.bool)
        elif d is not 0:                
            buffer = [struct.copy() for _ in range(d)]
            new_struct = np.stack(buffer, axis=0)
            return new_struct.astype(np.bool)
        
    def square(self, w):
        struct = np.zeros((w, w), dtype='uint8')
        x, y = np.indices((w, w))
        mask = (x - w) * (y - w) <= n**2
        struct[mask] = 1
        return struct.astype(np.bool)

    def cube(self, w):
        struct = np.zeros((2 * w + 1, 2 * w + 1, 2 * w + 1))
        x, y, z = np.indices((2 * w + 1, 2 * w + 1, 2 * w + 1))
        mask = (x - w) *  (y - w) * (z - w) <= w**3
        struct[mask] = 1
        return struct.astype(np.bool)

    def circle(self, r):
        struct = np.zeros((2 * r + 1, 2 * r + 1))
        x, y = np.indices((2 * r + 1, 2 * r + 1))
        mask = (x - r)**2 + (y - r)**2 <= r**2
        struct[mask] = 1
        return struct.astype(np.bool)

    def sphere(self, r):
        struct = np.zeros((2 * r + 1, 2 * r + 1, 2 * r + 1))
        x, y, z = np.indices((2 * r + 1, 2 * r + 1, 2 * r + 1))
        mask = (x - r)**2 + (y - r)**2 + (z - r)**2 <= r**2
        struct[mask] = 1
        return struct.astype(np.bool)

In [ ]:
def find_largest_component(filled_binary):
    """Find the largest connected component in the binary image.
    This function labels the binary image and finds the summation
    of pixels in the labeled regions. The regions with the max pixels
    are chosen. Method adopted from https://tinyurl.com/y3m4qdg4
    
    filled_binary: Binary image without holes
    """
    # Label the regions in the binary image.
    labeled_im, num_of_regions =  label(filled_binary)
    
    # Sum up all pixels in the regions of the labeled binary.
    sizes = sum(filled_binary, labeled_im, range(num_of_regions + 1))
    
    # Store indicies of the regions with the greatest
    # number of pixels.
    mask = sizes == max(sizes)
    
    largest_regions = mask[labeled_im]
    
    return largest_regions, num_of_regions

#### Unsharp mask
The following operation is conducted

$ \tilde{I}_L = I\dot(1+5w) - 5w\tilde{H}^L * I $

In [ ]:
def unsharp_mask(img, sigma, w, length=1):
    """Perform unsharp masking on the grayscale image given.
    This function performs the following calculation ...
    Ibar_l = I(1+5w) - 5w x Hbar_l * I
    
    img: grayscale image
    """
    # Convolve 1D filters twice to more efficiently
    # perform the equvalent of a 3 by 3 filter.
    Hbar_x = general_gaussian(M=2*length+1, p=1, sig=sigma)
    Hbar_y = Hbar_x.reshape((1, -1))[0]
    
    smoothed = convolve1d(img, Hbar_x, mode='constant')
    smoothed = convolve1d(smoothed, Hbar_y, mode='constant')
    
    # Perform scaling of the initial image.
    img_plus = img.copy() * (1 + 5 * w)
    
    # Store indicies of the regions with the greatest
    # number of pixels.
    smooth_plus = smoothed * 5 * w
    
    return img_plus - smooth_plus

In [ ]:
# file_loc = './fMRI_fMRI_(64x64)_FOV=2cm_TR=1s_20201109125634_4.nii'
file_loc = '/Users/nowusu/Documents/axelson_test/bold_7sl/NIfTI/bold_7sl_fMRI_(96x96)_FOV=2cm_20210120155254_8.nii'
# nii_files = glob(file_loc + '*.dcm')
nii_obj = load(file_loc)
nii_hdr = nii_obj.header

im_data = nii_obj.get_fdata()

In [ ]:
multi_slice_display(im_data[:,:,:,0])

In [ ]:
usm_result = unsharp_mask(im_data[:,:,:,0], sigma=0.5, w=0.2)

In [ ]:
multi_slice_display(usm_result)

In [ ]:
usm_result[usm_result < 0] = 0

In [ ]:
np.amax(usm_result[:,:,0])

In [ ]:
multi_slice_display(usm_result)

In [ ]:
sphere_stel = struct_el()
disk_H = sphere_stel.shape['sphere'](5)
footprints=generate_binary_structure(3, 1)

In [ ]:
eroded_im1 = grey_erosion(usm_result, footprint=footprints, structure=disk_H)

In [ ]:
multi_slice_display(eroded_im1)

In [ ]:
m, n, o, t = im_data.shape
threshold_im1 = np.ones((m, n, o), dtype='uint8')
threshold_im1[eroded_im1 < 500] = 0

### Hole-filling step

In [ ]:
square_stel = struct_el()
new_4neighbor = square_stel.shape['4neighbor'](1)
# new_4neighbor

In [ ]:
filled_holes = binary_fill_holes(threshold_im1, structure=new_4neighbor)
# filled_holes = binary_fill_holes(threshold_im1)
threshold1_complement = 1 - filled_holes

In [ ]:
multi_slice_display(filled_holes)

In [ ]:
multi_slice_display(threshold1_complement)

### Finding the largest connected component
code for this was adopted from ...
https://tinyurl.com/y62zqely

In [ ]:
largest_reg1, num_of_regions = find_largest_component(filled_holes)
num_of_regions

In [ ]:
multi_slice_display(largest_reg1)

### Binary Opening Step

In [ ]:
### Binary erosion of a binary union ###
union_im = threshold1_complement | largest_reg1
disk_H2 = sphere_stel.shape['sphere'](2)
bin_erode = binary_erosion(union_im, structure=disk_H2, iterations=1)

largest_reg2, num_of_regions2 = find_largest_component(bin_erode)
num_of_regions2

In [ ]:
### Binary dilation of a binary intersect ###
disk_H3 = sphere_stel.shape['cube'](1)
# im1 = np.where(largest_reg1 > 0, 1, 0)
im1 = np.where(threshold1_complement < 1, 1, 0)
im2 = np.where(bin_erode > 0, 1, 0)

intersect = im1 & im2
dilate_im2 = binary_dilation(intersect, disk_H3, iterations=2)

In [ ]:
multi_slice_display(dilate_im2)

In [ ]:
affine = nii_obj.affine
hdr = nii_obj.header
fmrIm = Nifti1Image(dilate_im2.astype(np.float32), affine)
fmrIm.header.set_qform(hdr.get_qform())

oldSform = hdr.get_sform(coded=True)
fmrIm.header.set_sform(oldSform[0],oldSform[1])

fmrIm.header.set_data_shape(hdr.get_data_shape())

save(fmrIm, filename='/Users/nowusu/Documents/axelson_test/bold_7sl/NIfTI/fmri_mask.nii.gz')